# Load libraries

In [23]:
import pandas as pd
import numpy as np
from scipy.stats import beta
import plotly.graph_objs as go
import copy
import random

# Create banner class

In [24]:
class banner_class():
    def __init__(self,Id,p):
        # Banner win probability
        self.p = p
        # Banner statistics and characteristics
        self.stat = {'id': Id,'a':0,'b':0,'conv':0,'p':self.p}
        # number of wins, number of losses
        self.a = 0
        self.b = 0

        
    # statistics collection function
    def get_stat(self,value):
        self.stat['b'] += 1
        if value == 1:
            self.stat['a'] +=1
        self.stat['conv'] = self.stat['a'] / self.stat['b']

        
    # conversion simulation function
    def conversion(self):
        if self.p > random.random():
            result = 1
        else:
            result = 0
        self.get_stat(result)

# Create a banner generator class

In [25]:
class banners_generator():

    def __init__(self,a, b):
        global random
        self.a = a
        self.b = b
        

    def generate(self,n):
        banners = [banner_class(i,random.uniform(self.a, self.b)) for i in range(n)]
        return banners

# EpsilonGreedy

In [26]:
class EpsilonGreedy():
    def __init__(self, data, n, epsilon = 0.15):
        self.epsilon = epsilon # probability of explore
        self.data = data
        self.n = n
        self.rc = [i for i in range(len(data))]
    

    # Function that returns the banner with the highest conversion
    def get_conv(self):
        l = []
        for i in self.data:
            l.append(i.stat['conv'])
        return np.argmax(l)


    def start_test(self):
        for _ in range(self.n):
            if random.random() > self.epsilon:
                self.data[self.get_conv()].conversion()
            else:
                self.data[random.choice(self.rc)].conversion()
        return self.get_result()

    
    def get_result(self):
        sort_data = sorted(self.data, key=lambda d: d.stat['conv'],reverse=True)
        sort_data  = pd.DataFrame([sort_data[i].stat for i in range(len(sort_data))])
        return pd.DataFrame([i.stat for i in self.data])

# Create a Thompson sampling class

In [27]:
class thompson_sampling():    


    def __init__(self,data,l,r,n):
        self.length = len(data)
        self.data = data
        self.win_index = None
        self.win_list = []
        self.l = l
        self.r = r
        self.z = 0.0001
        self.n = n
        

    # Function that simulates 1 game on the machine
    # the main part of the algorithm
    def do_sample(self):
        # Adding a winner to the list of winners
        self.win_list.append(self.win_index)
        # post a banner
        self.data[self.win_index].conversion()
        # Refine the distributions and display the winner
        self.conv_list = []
        # create a random number for each of the distributions
        # then take the maximum
        for i in range(self.length):
            conv = random.betavariate(1 + self.data[i].stat['a'], 
                                      1+ self.data[i].stat['b'])   
            self.conv_list.append(conv)
        self.win_index = np.argmax(self.conv_list)


    # Let's create a function for viewing indicators
    def print_data(self):
        result = [self.data[i].stat for i in range(self.length)]
        return pd.DataFrame(result)


    # create color for further rendering
    def create_color(self):
        colors = []
        for _ in range(self.length):
            r1 = np.random.randint(0,255)
            r2 = np.random.randint(0,255)
            r3 = np.random.randint(0,255)   
            color = f'rgba({r1},{r2},{r3},.3)'
            colors.append(color)
        return colors


    # This function is necessary to obtain distribution characteristics
    # It will be used in both visualization and testing.
    def get_distrib(self):
        x = np.arange(self.l,self.r,self.z)
        y_list = []
        self.quantiles = []
        for i in range(self.length):

            Id = self.data[i].stat['id']
            y = beta.pdf(x,1 + self.data[i].stat['a'],
                           1 + self.data[i].stat['b'])
            y_list.append(y)
        return y_list


    def start_test(self,plot = False):
        self.win_index = np.random.randint(0,self.length,1)[0]
        # Let's make the first game to get some kind of distribution
        self.do_sample()
        # Let's get the distribution
        self.get_distrib()
        # start sampling
        for _ in range(self.n):
            self.do_sample()
        # We get y distributions
        y_list = self.get_distrib()
        # Getting colors for distributions
        colors = self.create_color()
        x = np.arange(self.l,self.r,self.z)
        fig = go.Figure(data=[go.Scatter(x = x,
                        y = y_list[a],
                        marker = dict(color=(colors[a])),
                        fillcolor = colors[a],
                        fill='tozeroy') for a in range(self.length)])
        fig.update_layout(
        title="Conversion Distribution",
        xaxis_title="Conversion value",
        yaxis_title="Probability Density")
        if plot == True:
            fig.show()
        else:
            pass
        return self.get_result()


    # Getting test data
    def get_result(self):
        sort_data = sorted(self.data, key = lambda d: d.stat['conv'],reverse=True)
        sort_data  = pd.DataFrame([sort_data[i].stat for i in range(len(sort_data))])
        return pd.DataFrame([i.stat for i in self.data])

# A/B test

In [28]:
class ab_test_model():


    def __init__(self,data,n):
        self.data = data
        self.n = n // len(self.data)


    def start_test(self):
        for d in range(len(self.data)):
            for _ in range(self.n):
                self.data[d].conversion()
        return self.get_result()


    def get_result(self):
        sort_data = sorted(self.data, key=lambda d: d.stat['conv'],reverse=True)
        sort_data  = pd.DataFrame([sort_data[i].stat for i in range(len(sort_data))])
        return pd.DataFrame([i.stat for i in self.data])

# Simulation

In [29]:
class test_algo():

    def __init__(self,models,l,r):
        self.models = models
        self.l = l
        self.r = r
        
    def compare(self,query):
        statistic = []
        log = []
        # Get the required request
        for q in query:
            n,num_ban,attempt = q

            # Let's create a dictionary with test values
            results = {}
            for model in models:
                results[model.__name__] = 0

            # Results storage list
            for _ in range(attempt):
                # Generate data for each attempt
                data = banners_generator(self.l, self.r).generate(num_ban)
                # Iterating over the models
                for model in self.models:
                    copy_data = copy.deepcopy(data)

                    if model.__name__ == 'thompson_sampling':
                        result = model(copy_data, self.l, self.r, n).start_test()
                        results[model.__name__] += np.sum(result["a"])

                    elif model.__name__ == 'EpsilonGreedy':
                        result = model(copy_data, n, 0.15).start_test()
                        results[model.__name__] += np.sum(result["a"])

                    else:
                        result = model(copy_data, n).start_test()
                        results[model.__name__] += np.sum(result["a"])

            # For each query, we turn up the results into a dataframe
            results = pd.DataFrame([results]).T.sort_values(0,ascending=False)

            log.append(f'Query: n={n}, num_banners={num_ban}, attempt={attempt}, Победитель - {results.index[0]}')
            print(log[-1])

            statistic.append(results)
        display(self.stat(statistic))

    def stat(self,data):
        ft = pd.DataFrame()

        for i in data:
            ft = ft.append(i)
        
        ft = ft.reset_index()
        ft = ft.groupby('index').sum()
        ft = ft.sort_values(by = 0,ascending=False)
        ft = ft.reset_index()

        gap = np.array(ft[0])
        for i in range(1,len(gap)):
            gap[i] = gap[0] - gap[i]
        gap = gap / gap[0]
        gap[0] = 0
        ft['Difference_from_the_winner'] = gap

        ft.columns = ['Algorithm','number_of_wins','Difference_from_the_winner']
        return ft


In [30]:
models = [ab_test_model,EpsilonGreedy,thompson_sampling]

test = test_algo(models, 0.03, 0.5)

test.compare([
    [4000,5,1],
    [10000,10,1],
    [35000,30,1],
    [50000,50,1]
])

Query: n=4000, num_banners=5, attempt=1, Победитель - thompson_sampling
Query: n=10000, num_banners=10, attempt=1, Победитель - thompson_sampling
Query: n=35000, num_banners=30, attempt=1, Победитель - thompson_sampling
Query: n=50000, num_banners=50, attempt=1, Победитель - thompson_sampling


,Algorithm,number_of_wins,Difference_from_the_winner
0,thompson_sampling,45746,0.000000
1,EpsilonGreedy,45156,0.012897
2,ab_test_model,28482,0.377388


# Demonstration 

In [31]:
l = 0.03
r = 0.5
data = banners_generator(l, r).generate(5)

In [32]:
ts = thompson_sampling(data,l,r,1000)

In [33]:
ts.start_test(plot = True)

,id,a,b,conv,p
0,0,34,107,0.317757,0.451582
1,1,2,26,0.076923,0.113363
2,2,17,73,0.232877,0.255124
3,3,362,765,0.473203,0.493382
4,4,3,30,0.100000,0.040397
